## Multi Tooling with Foundry Agents

![lab_flow](./Assets/lab_flow.png)

### Installing Required Libraries

In [ ]:
%pip install azure-ai-projects==2.0.0b2 openai==1.109.1 python-dotenv azure-identity

### Setting Up the Environment Variables

In [ ]:
import os
from dotenv import load_dotenv
from azure.identity import DefaultAzureCredential
from azure.ai.projects import AIProjectClient
from azure.ai.projects.models import PromptAgentDefinition
import jsonref
from azure.ai.projects.models import PromptAgentDefinition, MCPTool, Tool

load_dotenv()

foundry_project_endpoint = os.getenv("FOUNDRY_PROJECT_ENDPOINT")
model_deployment_name = os.getenv("MODEL_DEPLOYMENT_NAME")
mcp_server_name = os.getenv("MCP_SERVER_NAME")

### Setting Up the Foundry Project Client

In [ ]:
project_client = AIProjectClient(
    endpoint=foundry_project_endpoint,
    credential=DefaultAzureCredential(),
)

### Creating the OpenAI Client

In [ ]:
openai_client = project_client.get_openai_client()

### Initializing the OpenAPI Tool

In [ ]:
with open("./weather_openapi.json", "r") as f:
        openapi_weather = jsonref.loads(f.read())

# Initialize agent OpenApi tool using the read in OpenAPI spec
weather_tool = {
        "type": "openapi",
        "openapi":{
            "name": "weather",
            "spec": openapi_weather,
            "auth": {
                "type": "anonymous"
            },
        }
}

### Getting the Connection ID for the MCP Server

In [ ]:
connection_id = ""

for connection in project_client.connections.list():
    if connection.name == mcp_server_name:
        connection_id = connection.id
        break

print(f"The MCP Server Connection ID is: {connection_id}")

### Creating the MCP Tool Spec

In [ ]:
mcp_tool = MCPTool(
    server_label = "microsoft_learn_server",
    server_url="https://learn.microsoft.com/api/mcp",
    require_approval="never",
    project_connection_id=connection_id
)

### Creating the Agent with Multiple Tools

In [ ]:
agent_name = "multi-tool-agent"

agent = project_client.agents.create_version(
    agent_name=agent_name,
    definition=PromptAgentDefinition(
        model=model_deployment_name,
        instructions="You are a helpful assistant that can use multiple tools to answer user queries.",
        tools=[weather_tool, mcp_tool],
    )
)

print(f"Created agent: {agent.name} with ID: {agent.id}")

### Creating a Conversation Object for the Agent Chat System

In [ ]:
# create a conversation to use with the agent
conversation = openai_client.conversations.create()
print(f"Created conversation with id: {conversation.id}")

### Calling our Agent

In [ ]:
user_query = "Let me know the weather in Seattle and also find me a Microsoft Learn module on Microsoft Foundry."

In [ ]:
response = openai_client.responses.create(
    conversation=conversation.id,
    extra_body = {
        "agent": {
            "name": agent_name,
            "type": "agent_reference"
        }
    },
    input = user_query
)

print(f"Agent response: {response.output_text}")